<a href="https://colab.research.google.com/github/arzharch/basic-RAG/blob/main/Rag_working_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import getpass

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  #document transformer: text splitter for chunking
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma #vector store
from langchain import HuggingFaceHub  #model hub
from langchain.chains import RetrievalQA

In [ ]:
os.environ['HUGGING_FACE_HUB_API_KEY'] = getpass.getpass('Hugging face api key:')

Hugging face api key:··········


In [ ]:
path = '/content/monopoly.pdf'
loader = PyPDFLoader(path)
pages = loader.load()

In [ ]:
len(  pages)

8

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = splitter.split_documents(pages)

In [ ]:
len(docs)

37

In [ ]:
embeddings = HuggingFaceEmbeddings()
doc_search = Chroma.from_documents(docs, embeddings)

In [ ]:
query = "how does the game end?"
similar_docs = doc_search.similarity_search(query, k=3)

In [ ]:
similar_docs

[Document(metadata={'page': 2, 'source': '/content/monopoly.pdf'}, page_content='all properties which it sells and auctions. \nThe Bank nwer "goes broke." If the Bank runs out of money, the Banker \nmay issue as much more as needed by writing on any ordinary paper. \nTHE PLAY: Starting with the Banker, each player in turn throws the dice. \nThe player with the highest total starts the play: Place your \ntoken on the corner marked "GO," throw the dice and move \nyour token in the direction of the arrow the number of \nspaces indicated by the dice. After you have completed'),
 Document(metadata={'page': 2, 'source': '/content/monopoly.pdf'}, page_content='all properties which it sells and auctions. \nThe Bank nwer "goes broke." If the Bank runs out of money, the Banker \nmay issue as much more as needed by writing on any ordinary paper. \nTHE PLAY: Starting with the Banker, each player in turn throws the dice. \nThe player with the highest total starts the play: Place your \ntoken on the

In [ ]:
repo_id = "tiiuae/falcon-7b"
llm = HuggingFaceHub(huggingfacehub_api_token = os.environ['HUGGING_FACE_HUB_API_KEY'],
                     repo_id=repo_id, model_kwargs={'temperature': 0.8, 'max_length':1000})

#you can modify the parameters temperature and max length to generate different responses

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
retrieval_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type='stuff',
    retriever=doc_search.as_retriever(),
    chain_type_kwargs={
        "memory":ConversationBufferMemory(memory_key="question") #Tell the memory to use the 'question' key
    }
)

In [ ]:
query = "give summary"
retrieval_chain.run(query)

'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nall properties which it sells and auctions. \nThe Bank nwer "goes broke." If the Bank runs out of money, the Banker \nmay issue as much more as needed by writing on any ordinary paper. \nTHE PLAY: Starting with the Banker, each player in turn throws the dice. \nThe player with the highest total starts the play: Place your \ntoken on the corner marked "GO," throw the dice and move \nyour token in the direction of the arrow the number of \nspaces indicated by the dice. After you have completed\n\nall properties which it sells and auctions. \nThe Bank nwer "goes broke." If the Bank runs out of money, the Banker \nmay issue as much more as needed by writing on any ordinary paper. \nTHE PLAY: Starting with the Banker, each player in turn throws the dice. \nThe player with the highest total starts the play: Place your \ntoke

In [ ]:
import gradio as gr

In [ ]:
# Define the Gradio interface
def qa_interface(query):
    response = retrieval_chain.run(query)
    return response

iface = gr.Interface(
    fn=qa_interface,
    inputs=gr.Textbox(lines=2, label="Enter your question"),
    outputs=gr.Textbox(label="Answer")
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://92dfd1ebd20032891b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
